In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import gc
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

/home/davidsolow/.pyenv/versions/3.11.1/envs/med-abbrev-mystery/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-12 18:55:04.893153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 18:55:04.906874: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 18:55:04.906899: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-12 18:55:04.917335: I tensorflow/core/platform/cpu_feature

Num GPUs Available:  1


2024-07-12 18:55:06.244941: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:55:06.248901: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:55:06.251889: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
max_len=200

def process_location(location):
    return list(map(int, eval(location)))[0]

def process_label(label):
    return eval(label)[0]

# Reading and processing training data
train_df = pd.read_csv('data/medal_train.csv')
train_df['location'] = train_df['location'].map(process_location)
train_df['label'] = train_df['label'].map(process_label)
train_df['abbreviation'] = train_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
train_df = train_df[train_df['location'] <= max_len-51]

# Reading and processing validation data
val_df = pd.read_csv('data/medal_val.csv')
val_df['location'] = val_df['location'].map(process_location)
val_df['label'] = val_df['label'].map(process_label)
val_df['abbreviation'] = val_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
val_df = val_df[val_df['location'] <= max_len-51]

# Reading and processing test data
test_df = pd.read_csv('data/medal_test.csv')
test_df['location'] = test_df['location'].map(process_location)
test_df['label'] = test_df['label'].map(process_label)
test_df['abbreviation'] = test_df.apply(lambda row: row['text'].split()[row['location']], axis=1)
test_df = test_df[test_df['location'] <= max_len-51]

train_df.head()

,Unnamed: 0,abstract_id,text,location,label,abbreviation
0,0,14145090,velvet antlers vas are commonly used in tradit...,63,transverse aortic constriction,TAC
1,1,1900667,the clinical features of our cases demonstrate...,85,hodgkins lymphoma,HD
2,2,8625554,ceftobiprole bpr is an investigational cephalo...,90,methicillinsusceptible s aureus,MSSA
3,3,8157202,we have taken a basic biologic RPA to elucidat...,26,parathyroid hormonerelated protein,PTHrP
5,5,4846741,gb virus c gbvc or HGV hgv is transmitted by t...,5,hepatitis g virus,HGV


In [3]:
train_text = train_df['text'].values[:10000]
train_abbrv = train_df['abbreviation'].values[:10000]
train_loc = train_df['location'].values[:10000]
train_label = train_df['label'].values[:10000]

val_text = val_df['text'].values[:1000]
val_abbrv = val_df['abbreviation'].values[:100]
val_loc = val_df['location'].values[:1000]
val_label = val_df['label'].values[:1000]

test_text = test_df['text'].values[:100]
test_abbrv = test_df['abbreviation'].values[:1000]
test_loc = test_df['location'].values[:1000]
test_label = test_df['label'].values[:1000]

del train_df
del val_df
del test_df
gc.collect()

train_text[:2]

array(['velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in culture

In [4]:
# acronyms = train_abbrv.tolist() + val_abbrv.tolist() + test_abbrv.tolist()
# expansions = train_label.tolist() + val_label.tolist() + test_label.tolist()
# expansions = [word for expansion in expansions for word in expansion.split()]
# new_terms = list(set(acronyms + expansions))

model_checkpoint = "Charangan/MedBERT"
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
# num_new_tokens = tokenizer.add_tokens(new_terms)
# print(f"Number of new tokens added: {num_new_tokens}")


In [5]:
def tokenize(texts, locations, expansions, tokenizer=tokenizer, max_len=max_len):
    input_ids = []
    token_type_ids = []
    attention_masks = []
    extended_texts = []
    encoded_labels = []

    for text, loc, expansion in zip(texts, locations, expansions):
        encoded_input = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='tf'
        )
        
        input_ids.append(encoded_input['input_ids'])
        token_type_ids.append(encoded_input['token_type_ids'])
        attention_masks.append(encoded_input['attention_mask'])

        # Labels
        label = text.split()
        label = label[:loc] + expansion.split() + label[loc+1:]
        label = ' '.join(label)
        extended_texts.append(label)
        encoded_output = tokenizer.encode_plus(
            label,
            add_special_tokens=False,
            max_length=max_len,
            padding='max_length',
            truncation=True,
            return_tensors='tf'
        )
        encoded_labels.append(encoded_output['input_ids'])

    input_ids = np.array(input_ids, dtype=np.int32).squeeze()
    token_type_ids = np.array(token_type_ids, dtype=np.int32).squeeze()
    attention_masks = np.array(attention_masks, dtype=np.int32).squeeze()
    encoded_labels = np.array(encoded_labels, dtype=np.int32).squeeze()

    print("First text:\n", texts[0])
    print("First location:", locations[0])
    print("First acronym:", texts[0].split()[locations[0]])
    print("First expansion:", expansions[0])
    print("First text with expansion:\n", extended_texts[0])
    decoded_tokens = tokenizer.convert_ids_to_tokens(encoded_labels[0])
    print("First label with expansion:\n", tokenizer.convert_tokens_to_string(decoded_tokens))

    return input_ids, token_type_ids, attention_masks, encoded_labels

In [6]:
print("Train----------------------------")
train_data = tokenize(train_text,
                      train_loc,
                      train_label)
train_input_ids = train_data[0]
train_token_type_ids = train_data[1]
train_attention_masks = train_data[2]
train_encoded_labels = train_data[3]

print("Val------------------------------")
val_data = tokenize(val_text,
                    val_loc,
                    val_label)
val_input_ids = val_data[0]
val_token_type_ids = val_data[1]
val_attention_masks = val_data[2]
val_encoded_labels = val_data[3]

print("Test-----------------------------")
test_data = tokenize(test_text,
                     test_loc,
                     test_label)
test_input_ids = test_data[0]
test_token_type_ids = test_data[1]
test_attention_masks = test_data[2]
test_encoded_labels = test_data[3]

del train_text
del train_loc
del train_label
del val_text
del val_loc
del val_label
del test_text
del test_loc
del test_label

gc.collect()

Train----------------------------


2024-07-12 18:57:29.585023: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:57:29.591836: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-12 18:57:29.596731: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

First text:
 velvet antlers vas are commonly used in traditional chinese medicine and invigorant and contain many PET components for health promotion the velvet antler peptide svap is one of active components in vas based on structural study the svap interacts with tgfÎ² receptors and disrupts the tgfÎ² pathway we hypothesized that svap prevents cardiac fibrosis from pressure overload by blocking tgfÎ² signaling SDRs underwent TAC tac or a sham operation T3 one month rats received either svap mgkgday or vehicle for an additional one month tac surgery induced significant cardiac dysfunction FB activation and fibrosis these effects were improved by treatment with svap in the heart tissue tac remarkably increased the expression of tgfÎ² and connective tissue growth factor ctgf ROS species C2 and the phosphorylation C2 of smad and ERK kinases erk svap inhibited the increases in reactive oxygen species C2 ctgf expression and the phosphorylation of smad and erk but not tgfÎ² expression in cu

In [7]:
input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
token_type_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="token_type_ids")
attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

bert_inputs = {'input_ids': input_ids,
               'token_type_ids': token_type_ids,
               'attention_mask': attention_mask}

medbert_model = TFBertModel.from_pretrained(model_checkpoint, from_pt=True)
#medbert_model.resize_token_embeddings(len(tokenizer))
medbert_model.trainable = True

bert_outputs = medbert_model(bert_inputs)
last_hidden_state = bert_outputs.last_hidden_state

num_labels = tokenizer.vocab_size
output = tf.keras.layers.Dense(num_labels, activation='softmax', name='output')(last_hidden_state)

model = tf.keras.Model(inputs=[input_ids,
                               token_type_ids,
                               attention_mask],
                               outputs=output)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask (InputLayer  [(None, 200)]                0         []                            
 )                                                                                                
                                                                                                  
 input_ids (InputLayer)      [(None, 200)]                0         []                            
                                                                                                  
 token_type_ids (InputLayer  [(None, 200)]                0         []                            
 )                                                                                                
                                                                                              

In [8]:
history = model.fit(
    [train_input_ids, train_token_type_ids, train_attention_masks],
    train_encoded_labels,
    validation_data=([val_input_ids, val_token_type_ids, val_attention_masks,],
                      val_encoded_labels, ),
    epochs=5,
    batch_size=16
)


Epoch 1/5


I0000 00:00:1720825122.086258   62228 service.cc:145] XLA service 0x78b0f161a590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720825122.086281   62228 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-12 18:58:42.093993: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-12 18:58:42.116170: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902
I0000 00:00:1720825122.163675   62228 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


625/625 [==============================] - 177s 230ms/step - loss: 6.3552 - accuracy: 0.2358 - val_loss: 5.2990 - val_accuracy: 0.3168
Epoch 2/5
625/625 [==============================] - 135s 216ms/step - loss: 4.7814 - accuracy: 0.3662 - val_loss: 4.3331 - val_accuracy: 0.4027
Epoch 3/5
625/625 [==============================] - 131s 210ms/step - loss: 3.9146 - accuracy: 0.4548 - val_loss: 3.7662 - val_accuracy: 0.4699
Epoch 4/5
625/625 [==============================] - 123s 198ms/step - loss: 3.2302 - accuracy: 0.5299 - val_loss: 3.4768 - val_accuracy: 0.4885
Epoch 5/5
625/625 [==============================] - 134s 214ms/step - loss: 2.7181 - accuracy: 0.5826 - val_loss: 3.3295 - val_accuracy: 0.5066


In [15]:
test_loss, test_accuracy = model.evaluate(
    [test_input_ids, test_token_type_ids, test_attention_masks],
    test_encoded_labels
)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

predictions = model.predict([test_input_ids, test_token_type_ids, test_attention_masks])
y_pred = np.argmax(predictions, axis=-1)
y_true = test_encoded_labels

print("First X Test:\n", tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(test_input_ids[0])))
print("First Y True:\n", tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_true[0])))
print("First Y Pred:\n", tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(y_pred[0])))

4/4 [==============================] - 1s 113ms/step - loss: 3.2621 - accuracy: 0.5181
Test Loss: 3.2620556354522705
Test Accuracy: 0.5181000232696533
4/4 [==============================] - 0s 101ms/step


2024-07-12 19:24:39.029432: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2319680000 exceeds 10% of free system memory.


First X Test:
 [CLS] we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until stable increase in RL at three times control in six reactive sheep group c was obtained they were then compared to the three nonreactive sheep group b and a control group of eight sheep exposed to saline only group a in terms of pulmonary CF tests and bronchoalveolar lavage bal analyses RL was cm holsec in group a in group b and in group c trapping volume FRC by plethysmography and by helium rebreathing technique was l in group a in group b and in group c UP resistance at PF did not differ between any two CG but UP resistance near residual volume was cm holsec in a in b and in c in bal total cells were x ml in a in b and in c macrophages in bal were in [SEP]
First Y True:
 we developed an animal model of chronic allergic airway disease by repeatedly exposing nine sheep to tracheal instillation of ascaris antigen until s